# TODOs: (before 13th sep ) (next friday)

1. Need to create a balanced dataloader and save it all in google drive along with perturbations + augmentations, dataloader will just load from google drive using os. Use train set and from val set create test set also. (tonight). save locally also which can just be uploaded as is. (maybe run over night if needed) (by 7th)
2. pytorch calls validation step first , maybe have to change the val epochs to batch size instead (when is it called, why is grad_fn not available)
 (check from here -> https://github.com/Lightning-AI/pytorch-lightning/issues/13948)
3.understand a bit more about what zero grad and loss backward doesinternall first (by 8th)
4. maybe don't use pytorch lightning at all,  just use pytorch entirely to control stuff (by 11th)


# About

This notebook attempts to finetune a resnet model to be more Robust by leveraging Projected Gradient Descent (PGD) in the two different ways:
1. include it as a part of the dataset
2. include the projected gradient as well while training along with the regular gradient from the loss function 

# 0. Importing required libraries


In [1]:
# !pip install lightning[extra]

In [2]:
import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import uuid
import random
from datasets import load_dataset
from torchvision.models import resnet50, ResNet50_Weights
from torch import nn
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
import torchvision.transforms as transforms
import torch
import torch.nn.functional as F
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from torchmetrics import Accuracy
from torchvision.transforms import Compose
from torchvision.transforms import ToTensor
from tqdm import tqdm
import copy

# 1. Setup

In [3]:
# Manual seed for reproducibility
SEED = 1234
torch.manual_seed(SEED)

# Params for PGD
ALPHA = 2/255
STEPS = 20
EPSILON = 8/255

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"device: {DEVICE}")

# model = resnet50(weights=ResNet50_Weights.DEFAULT)
# print("initialized the model")

device: cuda


## 2. Creating a finetuning dataset

Idea is to create a balanced finetuning dataset which is run only once to be saved onto the disk and then from there we can just create dataloaders on that for finetuning

### 2.1 Reusing the same PGD attacker class from before

In [4]:
class ResnetPGDAttacker:
    def __init__(self):
        '''
        The PGD attack on Resnet model.
        :param model: The resnet model on which we perform the attack
        :param dataloader: The dataloader loading the input data on which we perform the attack
        '''
        self.model = resnet50(weights=ResNet50_Weights.DEFAULT)
        self.loss_fn = nn.CrossEntropyLoss()
        self.adv_images = []
        self.labels = []
        self.eps = 0
        self.alpha = 0
        self.steps = 0
        self.acc = 0
        self.adv_acc = 0
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        # Nullify gradient for model params
        for p in self.model.parameters():
            p.requires_grad = False

    def pgd_attack(self, image, label, eps=None, alpha=None, steps=None):
        '''
        Create adversarial images for given batch of images and labels

        :param image: Batch of input images on which we perform the attack, size (BATCH_SIZE, 3, 224, 224)
        :param label: Batch of input labels on which we perform the attack, size (BATCH_SIZE)
        :return: Adversarial images for the given input images
        '''
        if eps is None:
            eps = self.eps
        if alpha is None:
            alpha = self.alpha
        if steps is None:
            steps = self.steps

        images = image.clone().detach().to(self.device)
        adv_images = images.clone()
        labels = label.clone().detach().to(self.device)

        # Starting at a uniformly random point within the eps ball
        random_noise = torch.zeros_like(adv_images).uniform_(-eps, eps)
        adv_images = adv_images + random_noise

        for _ in range(steps):
            # Enable gradient tracking for adversarial images
            adv_images.requires_grad = True

            # Get model predictions and apply softmax
            outputs = self.model(adv_images).softmax(1)

            # Calculate loss
            loss = self.loss_fn(outputs, labels)

            # Compute gradient wrt images
            grad = torch.autograd.grad(
                loss, adv_images, retain_graph=False, create_graph=False
            )[0]
            adv_images = adv_images.detach()

            # Gradient update
            adv_images = adv_images + alpha * grad.sign()  # Update adversarial images using the sign of the gradient

            # Projection step
            # Clamping the adversarial images to ensure they are within the L∞ ball of eps radius of original image
            adv_images = torch.clamp(adv_images, images - eps, images + eps)

            adv_images = adv_images.detach()

        return adv_images  # Return the generated adversarial images



### 2.2 Creating a finetuned dataset and saving it in disk

In [5]:
class FineTuneDatasetGenerator:
    def __init__(self, num_classes=1000, num_images_per_class=4, num_transforms=3, num_perturbations=2,
                 save_path="./dataset"
                 ):
        self.num_classes = num_classes
        self.num_images_per_class = num_images_per_class
        self.num_transforms = num_transforms
        self.num_perturbations = num_perturbations
        self.save_path = save_path
        
        # Create the save directory if it doesn't exist
        os.makedirs(self.save_path, exist_ok=True)
        weights = ResNet50_Weights.DEFAULT
        self.resnet_transform = weights.transforms()  #PIL -> tensor

        self.transformations = [
            transforms.Compose([
                transforms.RandomRotation(15)
            ]),
            transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(15)
            ]),
            transforms.Compose([
                transforms.RandomVerticalFlip(),
                transforms.RandomRotation(15)
            ])
        ]

        self.pgd_attacker = ResnetPGDAttacker()

        self.ds = load_dataset("ILSVRC/imagenet-1k", split="train", streaming=True, trust_remote_code=True)
        self.ds = self.ds.shuffle()
        print(f"Fine Tune Dataset Generator has been initialized.")

    def save_datapoint_with_augmentation_and_perturbations(self, img, label):
        images = []
        tensor_img = self.resnet_transform(img)

        #original image
        images.append(tensor_img)

        #augmentations (will be four)
        for _ in range(self.num_transforms):
            transformed_img = random.choice(self.transformations)(img)
            transformed_tensor_img = self.resnet_transform(transformed_img)
            images.append(transformed_tensor_img)

        #perturbations on all images thus far
        batch = list(
            map(
                lambda img: img.to("cpu"),
                images
            )
        )
        # Get a batch of images and create corresponding labels
        img_batch = torch.stack(batch)
        label_batch = torch.tensor([label] * len(img_batch))  # Repeat the label for the batch

        for _ in range(self.num_perturbations):
            # Generate random parameters for PGD attack
            random_eps = random.uniform(0.01, 0.3)
            random_alpha = random.uniform(0.01, 0.1)
            random_steps = random.randint(15, 20)

            # Perform the PGD attack
            perturbed_images = self.pgd_attacker.pgd_attack(img_batch,
                                                            label_batch,
                                                            eps=random_eps,
                                                            alpha=random_alpha,
                                                            steps=random_steps)

            for perturbed_img in perturbed_images:
                images.append(perturbed_img)

        #save this
        self.save_images(images, label)

    def generate(self):
        for label in tqdm(range(self.num_classes), desc="Fetching images for each class"):
            # Define the filter function for the current class label
            def filter_class(example):
                return example['label'] == label and example['image'].mode == 'RGB'

            # Load the dataset and filter for the current class
            ds = copy.deepcopy(self.ds)
            ds = ds.filter(filter_class)

            # Use take to get the desired number of images
            ds = ds.take(self.num_images_per_class)

            i = 0
            for data in ds:
                img = data["image"]
                self.save_datapoint_with_augmentation_and_perturbations(img, label)
                i += 1
                if i == self.num_images_per_class:
                    break

            print(
                f"Saved {self.num_images_per_class * (1 + self.num_transforms) * self.num_perturbations} images for class {label}."
            )

    def save_images(self, images, label):
        for idx, image in enumerate(images):
            img_id = str(uuid.uuid4())
            save_file = os.path.join(self.save_path, f"class_{label}_img_{img_id}.pt")
            torch.save(image, save_file)

        del images


# Example usage
fine_tune_gen = FineTuneDatasetGenerator(num_perturbations=2, num_transforms=2, num_images_per_class=2, num_classes=2)
fine_tune_gen.generate()


Fine Tune Dataset Generator has been initialized.


Fetching images for each class:   0%|          | 0/2 [04:12<?, ?it/s]


KeyboardInterrupt: 

### 2.3 Creating a Pytorch dataset and dataloaders over the saved data in the disk

In [11]:
from torch.utils.data import random_split


class FineTunedDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.image_files = os.listdir(root_dir)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load the image from the file
        image_file = self.image_files[idx]
        image = torch.load(os.path.join(self.root_dir, image_file))

        # Extract label from the filename
        label = int(os.path.basename(image_file).split("_")[1])  # e.g., "class_0_img_3.pt" -> label = 0

        return image, label

def get_dataloaders(root_dir, batch_size, train_val_test_split):
    dataset = FineTunedDataset(root_dir)

    train_size = int(len(dataset) * train_val_test_split[0])
    val_size = int(len(dataset) * train_val_test_split[1])
    test_size = len(dataset) - train_size - val_size

    train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    return train_loader, val_loader, test_loader


C:\Users\paras\AppData\Local\Temp\ipykernel_16792\808235905.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  image = torch.load(os.path.join(self.root_dir, image_file))


KeyboardInterrupt: 

# 3. Implementing Robust Resnet

In [16]:
class RobustResnet(LightningModule):
    def __init__(self, train_loader, val_loader, test_loader, learning_rate=1e-3):
        super().__init__()
        self.model = resnet50(weights=ResNet50_Weights.DEFAULT)
        self.loss_fn = nn.CrossEntropyLoss()
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.val_loader = val_loader
        self.learning_rate = learning_rate
        self.train_acc = Accuracy("multiclass", num_classes=1000)
        self.val_acc = Accuracy("multiclass", num_classes=1000)
        self.test_acc = Accuracy("multiclass", num_classes=1000)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, labels = batch['image'], batch['label']
        logits = self.model(images)
        ce_loss = self.loss_fn(logits, labels)
        # pgd_loss = self.compute_pgd_loss(images, labels)
        # loss = ce_loss + pgd_loss

        self.train_acc(logits, labels)
        self.log('train_ce_loss', ce_loss)
        # self.log('train_pgd_loss', pgd_loss)
        # self.log('train_loss', loss)
        self.log('train_acc', self.train_acc, on_step=False, on_epoch=True)
        # return loss
        return ce_loss

    # def validation_step(self, batch, batch_idx):
    #     images, labels = batch['image'], batch['label']
    #     logits = self.model(images)
    #     ce_loss = self.loss_fn(logits, labels)
    #     pgd_loss = self.compute_pgd_loss(images, labels)
    #     loss = ce_loss + pgd_loss
    #     self.val_acc(logits, labels)
    #     self.log('val_ce_loss', ce_loss, on_step=False, on_epoch=True)
    #     self.log('val_pgd_loss', pgd_loss, on_step=False, on_epoch=True)
    #     self.log('val_loss', loss, on_step=False, on_epoch=True)
    #     self.log('val_acc', self.val_acc, on_step=False, on_epoch=True)

    #TODO. this wont work as the labels will be -1 in this case
    # def test_step(self, batch, batch_idx):
    #     images, labels = batch
    #     logits = self.model(images)
    #     ce_loss = F.cross_entropy(logits, labels)
    #     pgd_loss = self.pgd_loss(images, labels)
    #     loss = ce_loss + pgd_loss
    #     acc = self.test_acc(logits, labels)
    #     self.log('test_ce_loss', ce_loss)
    #     self.log('test_pgd_loss', pgd_loss)
    #     self.log('test_loss', loss)
    #     self.log('test_acc', acc)

    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)

    def compute_pgd_loss(self, images, labels):
        # randomize pgd params
        eps = random.uniform(0.01, 0.3)
        alpha = random.uniform(0.01, 0.1)
        steps = random.randint(15, 20)
        
        # take a copy
        adv_images = images.clone().detach().requires_grad_(True).to(images.device)
        
        # Starting at a uniformly random point within the eps ball
        random_noise = torch.zeros_like(adv_images).uniform_(-eps, eps)
        adv_images = adv_images + random_noise
               
        # do randomized PGD process and compute loss
        for _ in range(steps):
            # Get model predictions
            outputs = self.model(adv_images)

            # Calculate loss
            pgd_loss = self.loss_fn(outputs, labels)

            # Compute gradient wrt adversarial images
            pgd_loss.backward(retain_graph=True)

            with torch.no_grad():
                # Gradient update
                adv_images = adv_images + alpha * adv_images.grad.sign()  # Update adversarial images using the sign of the gradient
    
                # Projection step
                # Clamping the adversarial images to ensure they are within the L∞ ball of eps radius of original image
                adv_images = torch.clamp(adv_images, images - eps, images + eps)
            
            # Clear the gradients for the next step
            adv_images.grad = None

        return pgd_loss

    def train_dataloader(self):
        return self.train_loader

    def val_dataloader(self):
        return self.val_loader

    def test_dataloader(self):
        return self.test_loader
    
#regular kind
path = "C:\\Parashara\\Projects\\NUS projects\\Sem3\\Trustworthy ML\\Assignment 1\\PGDAttack\\dataset"
train_loader, val_loader, test_loader = get_dataloaders(path, batch_size=2, train_val_test_split=[0.8, 0.1, 0.1])


# Create the classifier
classifier = RobustResnet(train_loader, val_loader, test_loader)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Monitor validation loss for saving the best model
    dirpath='checkpoints/',  # Directory where checkpoints will be saved
    filename='best-checkpoint',  # Checkpoint filename
    save_top_k=1,  # Save only the best model
    mode='min'  # Minimize the monitored metric
)

# Create a Trainer
#TODO.x change this to ensure that the training step is called first instead of the validation step. 
trainer = Trainer(
    max_epochs=3,
    accelerator='gpu',  # Specify the accelerator type
    devices=1,  # Use 1 GPU if available
    callbacks=[checkpoint_callback],  # Add the checkpoint callback
    log_every_n_steps=1,  # Log metrics every step
    enable_progress_bar=True,  # Enable progress bar for visibility
    inference_mode=False  # Added this line to allow gradients during validation
)

# Train the model
trainer.fit(classifier)



Loading train ILSVRC/imagenet-1k dataset...
Creating dataloader with 2 batches for split train each with size 1
Loading validation ILSVRC/imagenet-1k dataset...
Creating dataloader with 2 batches for split validation each with size 1
Loading test ILSVRC/imagenet-1k dataset...
Creating dataloader with 2 batches for split test each with size 1


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
C:\Users\paras\anaconda3\envs\trustworthyml\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:68: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | model     | ResNet             | 25.6 M | train
1 | loss_fn   | CrossEntropyLoss   | 0      | train
2 | train_acc | MulticlassAccuracy | 0      | train
3 | val_acc   | MulticlassAccuracy | 0      | train
4 | test_acc  | MulticlassAccuracy | 0      | train
---------------------------------------------------------
25.6 M    Trainable params
0         Non-trainable params
25.6 M    Total params
102.228   Total estimated model params size (MB)
155       Modules in train mode
0         Modules in eval mode
C:\Users\paras\ana

Training: |          | 0/? [00:00<?, ?it/s]

C:\Users\paras\anaconda3\envs\trustworthyml\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:384: `ModelCheckpoint(monitor='val_loss')` could not find the monitored key in the returned metrics: ['train_ce_loss', 'train_acc', 'epoch', 'step']. HINT: Did you call `log('val_loss', value)` in the `LightningModule`?
`Trainer.fit` stopped: `max_epochs=3` reached.


In [13]:
# Create dataloaders
# Robust kind
# train_loader = create_dataloader("train", batch_num=2, batch_size=1)
# val_loader = create_dataloader("validation", batch_num=2, batch_size=1)
# test_loader = create_dataloader("test", batch_num=2, batch_size=1)


# Create the classifier
classifier = RobustResnet(train_loader, val_loader, test_loader)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  # Monitor validation loss for saving the best model
    dirpath='checkpoints/',  # Directory where checkpoints will be saved
    filename='best-checkpoint',  # Checkpoint filename
    save_top_k=1,  # Save only the best model
    mode='min'  # Minimize the monitored metric
)

# Create a Trainer
trainer = Trainer(
    max_epochs=3,
    accelerator='gpu',  # Specify the accelerator type
    devices=1,  # Use 1 GPU if available
    callbacks=[checkpoint_callback],  # Add the checkpoint callback
    log_every_n_steps=1  # Log metrics every step
)

# Train the model
trainer.fit(classifier)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
C:\Users\paras\anaconda3\envs\trustworthyml\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:68: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | model     | ResNet             | 25.6 M | train
1 | loss_fn   | CrossEntropyLoss   | 0      | train
2 | train_acc | MulticlassAccuracy | 0      | train
3 | val_acc   | MulticlassAccuracy | 0      | train
4 | test_acc  | MulticlassAccuracy | 0      | train
---------------------------------------------------------
25.6 M    Trainable params
0         Non-trainable params
25.6 M    Total params
102.228   Total estimated model params size (MB)
155       Modules in train mode
0         Modules in eval mode
C:\Users\paras\ana

NameError: name 'exit' is not defined

In [ ]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

In [ ]:
# Test the model
trainer.test()

In [ ]:
'''
TODO:
1. check tensorboard here in the cell directly 
2. evaluate the evaluate code as the checkpointing and reloading of that model is not as straightforward as I thought it would be 
'''

# 5. Saving model & Evaluating Results

In [ ]:
def load_model_from_checkpoint(checkpoint_path):
    # Load the checkpoint
    checkpoint = torch.load(checkpoint_path)

    # Create a new instance of your model
    finetuned_model = RobustResnet()  # Initialize with the required parameters

    # Load the model weights from the checkpoint
    finetuned_model.load_state_dict(checkpoint['state_dict'])

    # Set the model to evaluation mode
    finetuned_model.eval()
    
    return finetuned_model

def evaluate_model(checkpoint_path, original_model, test_dataloader, device=DEVICE):
    # Load the fine-tuned model from the checkpoint
    fine_tuned_model = load_model_from_checkpoint(checkpoint_path)

    # Move models to the appropriate device
    original_model.to(device)
    fine_tuned_model.to(device)

    # Initialize accuracy metrics
    original_acc = Accuracy()
    fine_tuned_acc = Accuracy()

    # Evaluate the original model
    with torch.no_grad():
        for batch in test_dataloader:
            images, labels = batch
            images, labels = images.to(device), labels.to(device)

            # Original model predictions
            original_logits = original_model(images)
            original_acc(original_logits, labels)

            # Fine-tuned model predictions
            fine_tuned_logits = fine_tuned_model(images)
            fine_tuned_acc(fine_tuned_logits, labels)

    # Calculate accuracies
    original_accuracy = original_acc.compute()
    fine_tuned_accuracy = fine_tuned_acc.compute()

    print(f'Evaluation Original Model Accuracy: {original_accuracy:.4f}')
    print(f'Evaluation Fine-Tuned Model Accuracy: {fine_tuned_accuracy:.4f}')

    return original_accuracy, fine_tuned_accuracy